In [1]:
import datetime
import email
import imaplib
import mailbox
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import re
import glob
from numpy import nansum
from numpy import nanmean
import datetime
import functools
from datetime import datetime

In [2]:
import math
from scipy.optimize import brentq
Gamma=0.66

#----------------numerical solution--------------------------
# Tw + (es*(Tw))/𝛾 = Ta + ea/ 𝛾
# Ta air temperature 
# RH relative humidit
def TwNumerical(Ta,RH):
    SaturatedVPTa = 6.11* np.exp( (17.269*Ta) / (237.3+Ta) )
    VaporPressure = SaturatedVPTa*RH/100.0    
    def GetRoot(Tw):
        SaturatedVPTw = 6.11* np.exp( (17.269*Tw) / (237.3+Tw) )
        Function = Tw + SaturatedVPTw/Gamma-Ta-VaporPressure/Gamma
        return Function
    root,info = brentq(GetRoot,-20,50, full_output=True)
    return root


#----------------stull's formula----------------------
# use Stull's formula to calculate wet bulb temperature
# Ta air temperature 
# RH relative humidity
def Tw_Stull(Ta, RH):
    Tw_Stull=Ta*np.arctan(0.151977*(RH+8.313659)**0.5)+\
            np.arctan(Ta+RH)-np.arctan(RH-1.676331)+\
            np.arctan(0.023101*RH)*0.00391838*RH**(3./2.)-4.686035
    return Tw_Stull

def Clean_table_TW(df, st, tr):

    df['Lon'] = df['Lon'].str.replace('W','-').str.replace('E','').str.replace(' ','').astype(float)
    df['Lat'] = df['Lat'].str.replace('S','-').str.replace('N','').str.replace(' ','').astype(float)
    df['Temp'] = df['Temp'].str.replace('C','').str.replace(' ','').astype(float)
    df['Temp']=(df['Temp']-32)*5/9
    df['RH'] = df['RH'].str.replace('%','').str.replace(' ','').astype(float)
    df['Time']= pd.to_datetime(st[4:6]+'-'+st[-2:]+'-'+st[0:4]+ ' ' + df['Time'])
    #df['Biked']=np.nan
    #df['Biked'][0]=(df['Time'][df.shape[0]-1]-df['Time'][0]).seconds/60
    #df['Date']=np.nan
    #df['Date'][0]=str('Trip '+tr+' ('+st[4:6]+'-'+st[-2:]+'-'+st[0:4]+')')
    #df['Date_all']=str('Trip '+tr+' ('+st[4:6]+'-'+st[-2:]+'-'+st[0:4]+')')
    #df['Time']=df['Time'].replace(year=int(st[0:4]), month=int(st[4:6]), day=int(st[-2:]))
    df['Tw_Numerical']=df.apply(lambda x: TwNumerical(x['Temp'], x['RH']), axis=1)
    df['Tw_Stull']=Tw_Stull(df['Temp'], df['RH'])
    
    
    
    
    df = df.fillna('')
    return df

In [3]:
Dat_list=glob.glob('Bike study-6/*.txt')
Dat_list

['Bike study-6\\20191002-evening.txt',
 'Bike study-6\\20191002-morning.txt',
 'Bike study-6\\20191003-evening.txt',
 'Bike study-6\\20191003-morning.txt',
 'Bike study-6\\20191004-evening.txt',
 'Bike study-6\\20191004-morning.txt',
 'Bike study-6\\20191022-evening.txt',
 'Bike study-6\\20191022-morning.txt',
 'Bike study-6\\20191023-evening.txt',
 'Bike study-6\\20191023-morning.txt',
 'Bike study-6\\20191025-evening.txt',
 'Bike study-6\\20191025-morning.txt',
 'Bike study-6\\20191028-evening.txt',
 'Bike study-6\\20191028-morning.txt',
 'Bike study-6\\20191029-evening.txt',
 'Bike study-6\\20191029-morning.txt',
 'Bike study-6\\20191030-evening.txt',
 'Bike study-6\\20191030-morning.txt',
 'Bike study-6\\20191031-evening.txt',
 'Bike study-6\\20191031-morning .txt',
 'Bike study-6\\20191115.txt',
 'Bike study-6\\20191118.txt',
 'Bike study-6\\20191120-morning.txt',
 'Bike study-6\\20191120.txt',
 'Bike study-6\\20191122.txt',
 'Bike study-6\\20191125.txt',
 'Bike study-6\\20200224e

In [24]:
# Read csvs and clean data
Dat_all=[]
ii=1
for item in Dat_list:
    Dat=pd.read_csv(item, names=["Time", "Lon", "Lat", "ID","Temp","RH"]) 
    Dat_clean=Clean_table_TW(Dat, str(item)[13:21], "{:0>2}".format(str(ii)))
    Dat_all.append(Dat_clean)
    ii=ii+1


In [25]:
Dat_all

[                    Time       Lon       Lat     ID       Temp     RH  \
 0    2019-10-02 08:08:12 -72.90217  41.35806   AFF9  23.500000  79.62   
 1    2019-10-02 08:08:13 -72.90217  41.35806   AFF9  23.527778  79.54   
 2    2019-10-02 08:08:14 -72.90216  41.35806   AFF9  23.527778  79.46   
 3    2019-10-02 08:08:15 -72.90216  41.35806   AFF9  23.500000  79.38   
 4    2019-10-02 08:08:16 -72.90217  41.35805   AFF9  23.527778  79.34   
 ...                  ...       ...       ...    ...        ...    ...   
 3806 2019-10-02 17:41:11 -72.90223  41.35813   AFF9  27.238889  64.31   
 3807 2019-10-02 17:41:12 -72.90223  41.35813   AFF9  27.222222  64.29   
 3808 2019-10-02 17:41:13 -72.90223  41.35813   AFF9  27.222222  64.32   
 3809 2019-10-02 17:41:14 -72.90221  41.35813   AFF9  27.238889  64.27   
 3810 2019-10-02 17:41:15 -72.90221  41.35813   AFF9  27.238889  64.21   
 
       Tw_Numerical   Tw_Stull  
 0        20.923797  20.838650  
 1        20.939260  20.854587  
 2        2

In [26]:
Dat_all_trimmed=[]
for item in Dat_all:
    Temp=item.iloc[120:]
    Temp=Temp[:-120]
    Dat_all_trimmed.append(Temp)

In [27]:
Dat_all_trimmed

[                    Time       Lon       Lat     ID       Temp     RH  \
 120  2019-10-02 08:10:15 -72.90222  41.35799   AFF9  23.311111  80.65   
 121  2019-10-02 08:10:16 -72.90222  41.35799   AFF9  23.261111  80.65   
 122  2019-10-02 08:10:17 -72.90222  41.35799   AFF9  23.261111  80.65   
 123  2019-10-02 08:10:18 -72.90222  41.35799   AFF9  23.261111  80.67   
 124  2019-10-02 08:10:19 -72.90222  41.35799   AFF9  23.277778  80.68   
 ...                  ...       ...       ...    ...        ...    ...   
 3686 2019-10-02 17:39:09 -72.90203  41.35565   AFF9  27.361111  66.32   
 3687 2019-10-02 17:39:10 -72.90211  41.35569   AFF9  27.327778  66.40   
 3688 2019-10-02 17:39:11 -72.90214  41.35571   AFF9  27.327778  66.40   
 3689 2019-10-02 17:39:12 -72.90221  41.35577   AFF9  27.322222  66.42   
 3690 2019-10-02 17:39:13 -72.90226  41.35581   AFF9  27.322222  66.49   
 
       Tw_Numerical   Tw_Stull  
 120      20.882117  20.794577  
 121      20.834984  20.746489  
 122      2

In [28]:
#Concatanate into one data frame
result= pd.concat(Dat_all_trimmed, ignore_index=True)
result
##Remove duplicate datetimes
result=result.drop_duplicates(subset=['Time']).sort_values(['Time']).reset_index()
result

,index,Time,Lon,Lat,ID,Temp,RH,Tw_Numerical,Tw_Stull
0,0,2019-10-02 08:10:15,-72.90222,41.35799,AFF9,23.311111,80.65,20.882117,20.794577
1,1,2019-10-02 08:10:16,-72.90222,41.35799,AFF9,23.261111,80.65,20.834984,20.746489
2,2,2019-10-02 08:10:17,-72.90222,41.35799,AFF9,23.261111,80.65,20.834984,20.746489
3,3,2019-10-02 08:10:18,-72.90222,41.35799,AFF9,23.261111,80.67,20.837610,20.749146
4,4,2019-10-02 08:10:19,-72.90222,41.35799,AFF9,23.277778,80.68,20.854635,20.766505
...,...,...,...,...,...,...,...,...,...
105923,132492,2020-03-12 13:42:22,-72.90227,41.35583,AFF9,8.722222,60.49,5.429262,4.918723
105924,132493,2020-03-12 13:42:24,-72.90250,41.35608,AFF9,8.722222,60.51,5.431016,4.920496
105925,132494,2020-03-12 13:42:25,-72.90250,41.35608,AFF9,8.722222,60.55,5.434522,4.924041
105926,132495,2020-03-12 13:42:26,-72.90249,41.35615,AFF9,8.688889,60.57,5.407179,4.895791


In [29]:
## Add Biked and Date column
result['Date']=np.nan
result['Date_all']=np.nan
result['Biked']=np.nan
kk=0
for index, row in result.iterrows():
    result['Date_all'][index]=str(result['Time'][kk])  
    if (index<=len(result)-2):
        if(((result['Time'][index+1]-result['Time'][index]).seconds>=300)):
            result['Biked'][kk]=(result['Time'][index]-result['Time'][kk]).seconds/60
            result['Date'][kk]=str(result['Time'][kk])
            result['Date_all'][index]=str(result['Time'][kk])
            kk=index+1
    else:
        result['Biked'][kk]=(result['Time'][index]-result['Time'][kk]).seconds/60
        result['Date'][kk]=str(result['Time'][kk])
        result['Date_all'][index]=str(result['Time'][kk])
        kk=index+1
        #result['Biked'][index]=np.nan
        #result['Date'][index]=np.nan
#     if (index==0):
#         #result['Biked'][index]=np.nan
#         #result['Date'][index]=np.nan
#         result['Date_all'][index]=str(result['Time'][index])
#         result['Date'][index]=str(result['Time'][index])
#         result['Biked'][index]=(result['Time'][kk]-result['Time'][index]).seconds/60

C:\Users\tirth\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\tirth\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\tirth\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while w

In [30]:
#Fin=result[(result['Biked']<=240) | (result['Biked']).isnull()]
#Remove cases where someone biked for 0 time
Fin=result[(result['Biked']>=5) | (result['Biked']).isnull()]
Fin=Fin[['Time', 'Lon', 'Lat', 'ID', 'Temp', 'RH', 'Biked','Date','Date_all','Tw_Numerical','Tw_Stull']]
Fin

,Time,Lon,Lat,ID,Temp,RH,Biked,Date,Date_all,Tw_Numerical,Tw_Stull
0,2019-10-02 08:10:15,-72.90222,41.35799,AFF9,23.311111,80.65,37.35,2019-10-02 08:10:15,2019-10-02 08:10:15,20.882117,20.794577
1,2019-10-02 08:10:16,-72.90222,41.35799,AFF9,23.261111,80.65,NaN,NaN,2019-10-02 08:10:15,20.834984,20.746489
2,2019-10-02 08:10:17,-72.90222,41.35799,AFF9,23.261111,80.65,NaN,NaN,2019-10-02 08:10:15,20.834984,20.746489
3,2019-10-02 08:10:18,-72.90222,41.35799,AFF9,23.261111,80.67,NaN,NaN,2019-10-02 08:10:15,20.837610,20.749146
4,2019-10-02 08:10:19,-72.90222,41.35799,AFF9,23.277778,80.68,NaN,NaN,2019-10-02 08:10:15,20.854635,20.766505
...,...,...,...,...,...,...,...,...,...,...,...
105923,2020-03-12 13:42:22,-72.90227,41.35583,AFF9,8.722222,60.49,NaN,NaN,2020-03-12 13:20:58,5.429262,4.918723
105924,2020-03-12 13:42:24,-72.90250,41.35608,AFF9,8.722222,60.51,NaN,NaN,2020-03-12 13:20:58,5.431016,4.920496
105925,2020-03-12 13:42:25,-72.90250,41.35608,AFF9,8.722222,60.55,NaN,NaN,2020-03-12 13:20:58,5.434522,4.924041
105926,2020-03-12 13:42:26,-72.90249,41.35615,AFF9,8.688889,60.57,NaN,NaN,2020-03-12 13:20:58,5.407179,4.895791


In [36]:
Fin['Temp_norm'] = Fin.groupby('Date_all')['Temp'].transform('mean')
Fin['Temp_norm'] = Fin['Temp']-Fin['Temp_norm']
Fin['RH_norm'] = Fin.groupby('Date_all')['RH'].transform('mean')
Fin['RH_norm'] = Fin['RH']-Fin['RH_norm']
Fin['Tw_Numerical_norm'] = Fin.groupby('Date_all')['Tw_Numerical'].transform('mean')
Fin['Tw_Numerical_norm'] = Fin['Tw_Numerical']-Fin['Tw_Numerical_norm']
Fin

,Time,Lon,Lat,ID,Temp,RH,Biked,Date,Date_all,Tw_Numerical,Tw_Stull,Temp_norm,RH_norm,Tw_Numerical_norm
0,2019-10-02 08:10:15,-72.90222,41.35799,AFF9,23.311111,80.65,37.35,2019-10-02 08:10:15,2019-10-02 08:10:15,20.882117,20.794577,0.799504,-4.218506,0.224226
1,2019-10-02 08:10:16,-72.90222,41.35799,AFF9,23.261111,80.65,NaN,NaN,2019-10-02 08:10:15,20.834984,20.746489,0.749504,-4.218506,0.177094
2,2019-10-02 08:10:17,-72.90222,41.35799,AFF9,23.261111,80.65,NaN,NaN,2019-10-02 08:10:15,20.834984,20.746489,0.749504,-4.218506,0.177094
3,2019-10-02 08:10:18,-72.90222,41.35799,AFF9,23.261111,80.67,NaN,NaN,2019-10-02 08:10:15,20.837610,20.749146,0.749504,-4.198506,0.179719
4,2019-10-02 08:10:19,-72.90222,41.35799,AFF9,23.277778,80.68,NaN,NaN,2019-10-02 08:10:15,20.854635,20.766505,0.766171,-4.188506,0.196745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105923,2020-03-12 13:42:22,-72.90227,41.35583,AFF9,8.722222,60.49,NaN,NaN,2020-03-12 13:20:58,5.429262,4.918723,-0.698591,0.153363,-0.577750
105924,2020-03-12 13:42:24,-72.90250,41.35608,AFF9,8.722222,60.51,NaN,NaN,2020-03-12 13:20:58,5.431016,4.920496,-0.698591,0.173363,-0.575996
105925,2020-03-12 13:42:25,-72.90250,41.35608,AFF9,8.722222,60.55,NaN,NaN,2020-03-12 13:20:58,5.434522,4.924041,-0.698591,0.213363,-0.572489
105926,2020-03-12 13:42:26,-72.90249,41.35615,AFF9,8.688889,60.57,NaN,NaN,2020-03-12 13:20:58,5.407179,4.895791,-0.731924,0.233363,-0.599832


In [35]:
Fin['Temp_norm'][Fin['Temp_norm'] > Fin['Temp_norm'].quantile(0.95)] = np.nan
Fin['Temp_norm'][Fin['Temp_norm'] < Fin['Temp_norm'].quantile(0.05)] = np.nan
Fin['RH_norm'][Fin['RH_norm'] > Fin['RH_norm'].quantile(0.95)] = np.nan
Fin['RH_norm'][Fin['RH_norm'] < Fin['RH_norm'].quantile(0.05)] = np.nan
Fin['Tw_Numerical_norm'][Fin['Tw_Numerical_norm'] > Fin['Tw_Numerical_norm'].quantile(0.95)] = np.nan
Fin['Tw_Numerical_norm'][Fin['Tw_Numerical_norm'] < Fin['Tw_Numerical_norm'].quantile(0.05)] = np.nan

C:\Users\tirth\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\tirth\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\tirth\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so w

In [37]:
Temp_ind = Fin.groupby('Date_all')['Temp_norm'].cumcount() % 9
RH_ind = Fin.groupby('Date_all')['RH_norm'].cumcount() % 9
Tw_ind = Fin.groupby('Date_all')['Tw_Numerical_norm'].cumcount() % 9
Fin['Temp_norm'][Temp_ind != 0]=np.nan
Fin['RH_norm'][Temp_ind != 0]=np.nan
Fin['Tw_Numerical_norm'][Temp_ind != 0]=np.nan
Fin

C:\Users\tirth\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\tirth\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\tirth\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Time,Lon,Lat,ID,Temp,RH,Biked,Date,Date_all,Tw_Numerical,Tw_Stull,Temp_norm,RH_norm,Tw_Numerical_norm
0,2019-10-02 08:10:15,-72.90222,41.35799,AFF9,23.311111,80.65,37.35,2019-10-02 08:10:15,2019-10-02 08:10:15,20.882117,20.794577,0.799504,-4.218506,0.224226
1,2019-10-02 08:10:16,-72.90222,41.35799,AFF9,23.261111,80.65,NaN,NaN,2019-10-02 08:10:15,20.834984,20.746489,NaN,NaN,NaN
2,2019-10-02 08:10:17,-72.90222,41.35799,AFF9,23.261111,80.65,NaN,NaN,2019-10-02 08:10:15,20.834984,20.746489,NaN,NaN,NaN
3,2019-10-02 08:10:18,-72.90222,41.35799,AFF9,23.261111,80.67,NaN,NaN,2019-10-02 08:10:15,20.837610,20.749146,NaN,NaN,NaN
4,2019-10-02 08:10:19,-72.90222,41.35799,AFF9,23.277778,80.68,NaN,NaN,2019-10-02 08:10:15,20.854635,20.766505,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105923,2020-03-12 13:42:22,-72.90227,41.35583,AFF9,8.722222,60.49,NaN,NaN,2020-03-12 13:20:58,5.429262,4.918723,NaN,NaN,NaN
105924,2020-03-12 13:42:24,-72.90250,41.35608,AFF9,8.722222,60.51,NaN,NaN,2020-03-12 13:20:58,5.431016,4.920496,NaN,NaN,NaN
105925,2020-03-12 13:42:25,-72.90250,41.35608,AFF9,8.722222,60.55,NaN,NaN,2020-03-12 13:20:58,5.434522,4.924041,NaN,NaN,NaN
105926,2020-03-12 13:42:26,-72.90249,41.35615,AFF9,8.688889,60.57,NaN,NaN,2020-03-12 13:20:58,5.407179,4.895791,NaN,NaN,NaN


In [38]:
Fin

,Time,Lon,Lat,ID,Temp,RH,Biked,Date,Date_all,Tw_Numerical,Tw_Stull,Temp_norm,RH_norm,Tw_Numerical_norm
0,2019-10-02 08:10:15,-72.90222,41.35799,AFF9,23.311111,80.65,37.35,2019-10-02 08:10:15,2019-10-02 08:10:15,20.882117,20.794577,0.799504,-4.218506,0.224226
1,2019-10-02 08:10:16,-72.90222,41.35799,AFF9,23.261111,80.65,NaN,NaN,2019-10-02 08:10:15,20.834984,20.746489,NaN,NaN,NaN
2,2019-10-02 08:10:17,-72.90222,41.35799,AFF9,23.261111,80.65,NaN,NaN,2019-10-02 08:10:15,20.834984,20.746489,NaN,NaN,NaN
3,2019-10-02 08:10:18,-72.90222,41.35799,AFF9,23.261111,80.67,NaN,NaN,2019-10-02 08:10:15,20.837610,20.749146,NaN,NaN,NaN
4,2019-10-02 08:10:19,-72.90222,41.35799,AFF9,23.277778,80.68,NaN,NaN,2019-10-02 08:10:15,20.854635,20.766505,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105923,2020-03-12 13:42:22,-72.90227,41.35583,AFF9,8.722222,60.49,NaN,NaN,2020-03-12 13:20:58,5.429262,4.918723,NaN,NaN,NaN
105924,2020-03-12 13:42:24,-72.90250,41.35608,AFF9,8.722222,60.51,NaN,NaN,2020-03-12 13:20:58,5.431016,4.920496,NaN,NaN,NaN
105925,2020-03-12 13:42:25,-72.90250,41.35608,AFF9,8.722222,60.55,NaN,NaN,2020-03-12 13:20:58,5.434522,4.924041,NaN,NaN,NaN
105926,2020-03-12 13:42:26,-72.90249,41.35615,AFF9,8.688889,60.57,NaN,NaN,2020-03-12 13:20:58,5.407179,4.895791,NaN,NaN,NaN


In [39]:
Fin.to_csv('Compiled_Sara_norm.csv')

In [40]:
ss=Fin.groupby('Date').max()
Final=pd.DataFrame(ss.index)

Final[Final['Date']==""]=np.nan
Final=Final.dropna()
Final

,Date
0,2019-10-02 08:10:15
1,2019-10-02 17:13:48
2,2019-10-03 08:14:05
3,2019-10-03 16:57:48
4,2019-10-04 08:13:33
5,2019-10-22 08:26:21
6,2019-10-22 17:12:18
7,2019-10-23 17:07:27
8,2019-10-28 17:19:56
9,2019-10-29 08:24:29


In [41]:
Final.to_csv('Compiled_Sara_Date.csv')

In [42]:
Fin

,Time,Lon,Lat,ID,Temp,RH,Biked,Date,Date_all,Tw_Numerical,Tw_Stull,Temp_norm,RH_norm,Tw_Numerical_norm
0,2019-10-02 08:10:15,-72.90222,41.35799,AFF9,23.311111,80.65,37.35,2019-10-02 08:10:15,2019-10-02 08:10:15,20.882117,20.794577,0.799504,-4.218506,0.224226
1,2019-10-02 08:10:16,-72.90222,41.35799,AFF9,23.261111,80.65,NaN,NaN,2019-10-02 08:10:15,20.834984,20.746489,NaN,NaN,NaN
2,2019-10-02 08:10:17,-72.90222,41.35799,AFF9,23.261111,80.65,NaN,NaN,2019-10-02 08:10:15,20.834984,20.746489,NaN,NaN,NaN
3,2019-10-02 08:10:18,-72.90222,41.35799,AFF9,23.261111,80.67,NaN,NaN,2019-10-02 08:10:15,20.837610,20.749146,NaN,NaN,NaN
4,2019-10-02 08:10:19,-72.90222,41.35799,AFF9,23.277778,80.68,NaN,NaN,2019-10-02 08:10:15,20.854635,20.766505,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105923,2020-03-12 13:42:22,-72.90227,41.35583,AFF9,8.722222,60.49,NaN,NaN,2020-03-12 13:20:58,5.429262,4.918723,NaN,NaN,NaN
105924,2020-03-12 13:42:24,-72.90250,41.35608,AFF9,8.722222,60.51,NaN,NaN,2020-03-12 13:20:58,5.431016,4.920496,NaN,NaN,NaN
105925,2020-03-12 13:42:25,-72.90250,41.35608,AFF9,8.722222,60.55,NaN,NaN,2020-03-12 13:20:58,5.434522,4.924041,NaN,NaN,NaN
105926,2020-03-12 13:42:26,-72.90249,41.35615,AFF9,8.688889,60.57,NaN,NaN,2020-03-12 13:20:58,5.407179,4.895791,NaN,NaN,NaN
